In [1]:
import yfinance as yf
import pandas as pd

def fetch_financial_data(ticker):
    stock = yf.Ticker(ticker)
    financials = stock.financials.transpose()
    balance_sheet = stock.balance_sheet.transpose()
    history = stock.history(period="20y")
    return financials, balance_sheet, history

def calculate_ratios(financials, balance_sheet, history, shares_outstanding):
    # Check if 'Intangible Assets' column exists
    if 'Net Tangible Assets' not in balance_sheet.columns:
        balance_sheet['Net Tangible Assets'] = balance_sheet['Total Assets'] - balance_sheet['Intangible Assets'] - balance_sheet['Total Liabilities']
    
    # Calculate ROTA (Return on Tangible Assets)
    financials['ROTA'] = financials['Net Income'] / balance_sheet['Net Tangible Assets']

    financials['ROE'] = financials['Net Income'] / balance_sheet['Stockholders Equity']

    # Calculate historical P/E ratio
    pe_ratios = []
    for year in financials.index.year:
        year_data = history.loc[history.index.year == year]
        if not year_data.empty:
            closing_price = year_data['Close'].iloc[-1]  # Use the closing price of the last month of the year
            earnings_per_share = financials.loc[financials.index.year == year, 'Net Income'].values[0] / shares_outstanding
            pe_ratio = closing_price / earnings_per_share
            pe_ratios.append(pe_ratio)
        else:
            pe_ratios.append(None)
    
    financials['P/E Ratio'] = pe_ratios

    return financials

def main():
    ticker = "BV"
    financials, balance_sheet, history = fetch_financial_data(ticker)
    shares_outstanding = yf.Ticker(ticker).info['sharesOutstanding']
    financials = calculate_ratios(financials, balance_sheet, history, shares_outstanding)

    # Display the financials with the calculated ROTA and P/E ratios
    print(financials[['Net Income', 'ROTA', 'ROE', 'P/E Ratio']])

if __name__ == "__main__":
    main()


            Net Income      ROTA       ROE   P/E Ratio
2024-09-30  66400000.0 -0.201762  0.037253   22.997665
2023-09-30  -7700000.0   0.01868 -0.004421 -104.429871
2022-09-30  14000000.0 -0.014488  0.011506   46.999642
2021-09-30  46300000.0 -0.057466  0.034483   29.041900
